## Лекция 4 - Словари и сортировка подсчётом
[Link](https://youtu.be/Nb5mW1yWVSs])

## Сортировка подсчетом
> Необходимо отсортировать массив из N целых чисел, каждое от 0 до К.

Считаем кол-во вхождений каждого числа, а затем выводить каждое число столько раз, сколько оно встречалось. Это займет O(N+K) и O(K) дополнительной памяти.

In [8]:
# функция изменяет значения последовательности
def countsort(seq):
    minval = min(seq)
    maxval = max(seq)
    k = (maxval - minval + 1)
    count = [0] * k
    for now in seq:
        count[now - minval] += 1
    nowpos = 0
    for val in range(0, k):
        for i in range(count[val]):
            seq[nowpos] = val + minval
            nowpos += 1
    return seq
            

In [9]:
countsort([5,5,5,4,1,3,2,2,3])

[1, 2, 2, 3, 3, 4, 5, 5, 5]

### Задача 1
> Дано два числа X и Y без ведущих нулей <br/>

Необходимо проверить, можно ли получить первое из второго перестановкой цифр

### Решение
Посчитаем кол-во вхождений каждой цифры в каждое из числе и сравним. Цифры будем постепенно добывать из числа справа с помощью %10 и //10

In [17]:
def isdigitpermutation(x, y):
    def countdigits(num):
        digitcount = [0] * 10
        while num > 0:
            lastdigit = num % 10
            digitcount[lastdigit] += 1
            num //= 10
        return digitcount
    
    digitsx = countdigits(x)
    digitsy = countdigits(y)
    for digit in range(10):
        if digitsx[digit] != digitsy[digit]:
            return False
    return True

In [21]:
isdigitpermutation(2021, 2120)

True

In [23]:
countdigits(2021)

[1, 1, 2, 0, 0, 0, 0, 0, 0, 0]

## Словари
* Словари - как множество, но к каждому ключу приписано значение
* Искать по значению в словаре нельзя
* Константа в сложности словарей заметно больше, чем у массивов, поэтому где можно - лучше использовать сортировку подсчетом
* Сортировку подсчетом неразумно использовать, если данные разреженные

### Задача 2
> На шахматной доске NxN находятся M ладей <br/>

Определите, сколько пар ладей бьют друг друга <br/>
$ 1 \le N \le 10^9$,$ 0 \le M \le 2*10^5$

### Решение
Для каждой занятой горизонтали и вертикали будем хранить количество ладей на них. Количество пар в горизонтали(вертикали) равно количеству ладей мину 1. Суммируем это количество пар для всех горизонталей и вертикалей.

In [24]:
def countbeatingrooks(rookcoords):
    
    def addrook(roworcol, key):
        if key not in roworcol:
            roworcol[key] = 0
        roworcol[key] += 1
        
    def countpairs(roworcol):
        pairs = 0
        for key in roworcol:
            pairs += roworcol[key] - 1
        return pairs
    
    rooksinrow = {}
    rooksincol = {}
    for row, col in rookcoords:
        addrook(rooksinrow, row)
        addrook(rooksincol, col)
    return countpairs(rooksinrow) + countpairs(rooksincol)

### Задача 3
> Дана строка S <br/>

Выведите гистограмму по сортированным символам с частотами в виде #

### Решение
Для каждого симлова в словаре посчитаем, сколько раз он встречался. Найдем самый частый символ и переберем количество от этого числа до 1. Пройдем по всем отсортированным ключам и если количество больше счетчика - выведем #.

In [26]:
def printchart(s):
    symcount = {}
    maxsymcount = 0
    for sym in s:
        if sym not in symcount:
            symcount[sym] = 0
        symcount[sym] += 1
        maxsymcount = max(maxsymcount, symcount[sym])
    sorteduniqsyms = sorted(symcount.keys())
    for row in range(maxsymcount, 0, -1):
        for sym in sorteduniqsyms:
            if symcount[sym] >= row:
                print('#', end = '')
            else:
                print(' ', end = '')
        print()
    print(''.join(sorteduniqsyms))

In [29]:
printchart('ah shit, here we go again')

#            
#            
# ## #       
# #####      
#############
 ,aeghinorstw


## Всегда ли асимптотически лучшее решение лучше?


При  $N>2^{500}$ решение за O(N) лучше,чем решение за O(N logN)

Другие критерии качества алгоритма:
- потребление памяти
- время на реализацию
- сложность поддержки
- возможность распараллеливания
- необходимая квалификация сотрудника
- стоимость оборудования

### Задача 4
> Сгруппировать слова по общим буквам <br/>


### Решение
Отсортируем в каждом слове буквы и это будет выступать в роли ключа, а значением будет список слов

In [40]:
def groupwords(words):
    groups = {}
    for word in words:
        sortedword = ''.join(sorted(word))
        if sortedword not in groups:
            groups[sortedword] = []
        groups[sortedword].append(word)
    ans = []
    for sortedword in groups:
        ans.append(groups[sortedword])
    return ans

In [41]:
groupwords(['eat', 'tea', 'tan', 'ate', 'nat', 'bat'])

[['eat', 'tea', 'ate'], ['tan', 'nat'], ['bat']]

Но если слова будут длинными - сортировка займет O(N logN).
Количесвто различных букв в слове $K \le N$, можем посчитать количество каждой за O(N) и отсортировать за O(K logK). <br/>
Попробуем оптимизировать?

In [45]:
def groupwords_opt(words):
    
    def keybyword(word):
        symcnt = {}
        for sym in word:
            if sym not in symcnt:
                symcnt[sym] = 0
            symcnt[sym] += 1
        lst = []
        for sym in sorted(symcnt.keys()):
            lst.append(sym)
            lst.append(str(symcnt[sym]))
        return ''.join(lst)
    
    groups = {}
    for word in words:
        groupkey = keybyword(word)
        if groupkey not in groups:
            groups[groupkey] = []
        groups[groupkey].append(word)
    ans = []
    for groupkey in groups:
        ans.append(groups[groupkey])
    return ans

In [46]:
groupwords_opt(['eat', 'tea', 'tan', 'ate', 'nat', 'bat'])

[['eat', 'tea', 'ate'], ['tan', 'nat'], ['bat']]

На самом деле работает дольше

### Домашнее задание
[Link](https://contest.yandex.ru/contest/28970)

In [1]:
# A. Толя-Карп и новый набор структур, часть 2

In [2]:
# B. Выборы в США

In [3]:
# C. Частотный анализ

In [4]:
# D. Выборы Государственной Думы

In [ ]:
# E. Форум